<a href="https://colab.research.google.com/github/Razanhus/t5/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install --upgrade ultralytics


In [ ]:
! pip install opencv-python ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
#loading yolo+libraries
import numpy as np
from ultralytics import YOLO
import cv2


In [111]:
model = YOLO('yolov8n.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [112]:
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video.")
    print("Video source:", 'input_video.mp4')

Error: Could not open video.
Video source: input_video.mp4


In [113]:
import os

video_source = 'input_video.mp4'  # Adjust as needed
if not os.path.exists(video_source):
    print("Error: Video file does not exist.")

Error: Video file does not exist.


In [ ]:
cap.isOpened()

True

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
#video info
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))



## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
#video writer for storing the output vid
output_path = 'YOLO_OUTPUT_VID2.mp4'
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


In [ ]:
#Initialize counters
left_count = 0
right_count = 0
top_count = 0
bottom_count = 0
#defining the line via counting
counting_line = int(width/4)

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [108]:
# Initialize video capture
cap = cv2.VideoCapture('input_video.mp4')  # Replace with your video source

# Read a single frame
ret, frame = cap.read()
if not ret:
    print("Failed to capture frame.")
else:
    # Initialize counting variables
    left_count = right_count = top_count = bottom_count = 0
    counting_line = frame.shape[1] // 4  # Example line for counting

    # Assuming 'model' is already loaded
    results = model(frame)  # Model inference

    for result in results:
        for detection in result.boxes:
            x1, y1, x2, y2 = map(int, detection.xyxy[0])
            conf = detection.conf[0]
            class_id = int(detection.cls[0])

            if class_id == 2:  # Assuming class_id 2 corresponds to 'Car'
                color = (0, 255, 0)
                label = f'Car {conf:.2f}'
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                center_x = (x1 + x2) // 2
                if center_x < counting_line:
                    left_count += 1
                elif center_x > 3 * counting_line:
                    right_count += 1
                elif center_x > counting_line and center_x < 2 * counting_line:
                    top_count += 1
                elif center_x > 2 * counting_line and center_x < 3 * counting_line:
                    bottom_count += 1

    # Draw counting lines
    for i in range(1, 4):
        cv2.line(frame, (i * counting_line, 0), (i * counting_line, frame.shape[0]), (255, 0, 0), 2)

    # Display counts on the frame
    cv2.putText(frame, f'Left: {left_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(frame, f'Right: {right_count}', (frame.shape[1] - 150, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(frame, f'Top: {top_count}', (frame.shape[1] // 2 - 50, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(frame, f'Bottom: {bottom_count}', (frame.shape[1] // 2 - 50, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Show the frame with detections
    cv2.imshow("Frame", frame)
    cv2.waitKey(0)  # Wait for a key press to close the window

# Release the video capture
cap.release()
cv2.destroyAllWindows()

Failed to capture frame.


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
#releasing resources:
cap.release()
out.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video